In [5]:
import numpy as np
import gym

In [ ]:

env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=True)

class FrozenLake:
    def __init__(self, env):
        self.env = env
        # Número de estados y acciones
        self.num_estados = self.env.observation_space.n
        self.num_acciones = self.env.action_space.n
        self.P = self.env.P

    def get_estados(self):
        return list(range(self.num_estados))

    def get_acciones(self):
        return list(range(self.num_acciones))

    def get_dinamica(self, estado, accion):
        return self.P[estado][accion]

    def MDP(self):
        print("Conjunto de estados (0 a {}):".format(self.num_estados - 1))
        print(self.get_estados())
        print("\nConjunto de acciones (0: Arriba, 1: Abajo, 2: Izquierda, 3: Derecha):")
        print(self.get_acciones())
        print("\nDinámica:")
        for estado in self.get_estados():
            for accion in self.get_acciones():
                dinamica = self.get_dinamica(estado, accion)
                print(f"\nDesde el estado {estado} con acción {accion}:")
                for prob, sig_estado, recompensa, terminal in dinamica:
                    print(f"  - Proba: {prob:.2f}, siguiente estado: {sig_estado}, recompensa: {recompensa}, ¿Acaba?: {terminal}")

if __name__ == "__main__":
    mdp = FrozenLake(env)
    mdp.MDP()



Conjunto de estados (0 a 15):
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

Conjunto de acciones (0: Arriba, 1: Abajo, 2: Izquierda, 3: Derecha):
[0, 1, 2, 3]

Dinámica:

Desde el estado 0 con acción 0:
  - Proba: 0.333, siguiente estado: 0, recompensa: 0.0, ¿Acaba?: False
  - Proba: 0.333, siguiente estado: 0, recompensa: 0.0, ¿Acaba?: False
  - Proba: 0.333, siguiente estado: 4, recompensa: 0.0, ¿Acaba?: False

Desde el estado 0 con acción 1:
  - Proba: 0.333, siguiente estado: 0, recompensa: 0.0, ¿Acaba?: False
  - Proba: 0.333, siguiente estado: 4, recompensa: 0.0, ¿Acaba?: False
  - Proba: 0.333, siguiente estado: 1, recompensa: 0.0, ¿Acaba?: False

Desde el estado 0 con acción 2:
  - Proba: 0.333, siguiente estado: 4, recompensa: 0.0, ¿Acaba?: False
  - Proba: 0.333, siguiente estado: 1, recompensa: 0.0, ¿Acaba?: False
  - Proba: 0.333, siguiente estado: 0, recompensa: 0.0, ¿Acaba?: False

Desde el estado 0 con acción 3:
  - Proba: 0.333, siguiente estado: 1, recompensa

In [7]:
def iteracion_valor(env, gamma=0.99, epsilon=1e-6, max_iter=1000): # se obtiene de ver las recompensas de cada estado dada cierta accion
    num_estados = env.observation_space.n
    num_acciones = env.action_space.n
    V = np.zeros(num_estados)
    
    for iteracion in range(max_iter):
        delta = 0
        V_nuevo = np.copy(V) 
        for s in range(num_estados):
            valores_accion = np.zeros(num_acciones) 
            for a in range(num_acciones):
                for transicion in env.P[s][a]:
                    probabilidad, s_sig, recompensa, terminal = transicion
                    valores_accion[a] += probabilidad * (recompensa + gamma * V[s_sig])
            V_nuevo[s] = np.max(valores_accion)
            delta = max(delta, np.abs(V_nuevo[s] - V[s]))
        V = V_nuevo
        if delta < epsilon:
            print(f'Convergencia alcanzada en {iteracion+1} iteraciones.')
            break
    return V

def iteracion_politica(env, V, gamma=0.99): #de la función de valor se extrae la política óptima.
    num_estados = env.observation_space.n
    num_acciones = env.action_space.n
    politica = np.zeros(num_estados, dtype=int)
    
    for s in range(num_estados):
        valores_accion = np.zeros(num_acciones)
        for a in range(num_acciones):
            for transicion in env.P[s][a]:
                probabilidad, s_sig, recompensa, terminal = transicion
                valores_accion[a] += probabilidad * (recompensa + gamma * V[s_sig])
        politica[s] = np.argmax(valores_accion)
    return politica

if __name__ == '__main__':
    gamma = 0.99
    epsilon = 1e-6
    V_optimo = iteracion_valor(env, gamma=gamma, epsilon=epsilon)
    print("valores:")
    print(V_optimo)
    
    politica_optima = iteracion_politica(env, V_optimo, gamma=gamma)
    print("Política óptima (por estado):")
    print(politica_optima)

Convergencia alcanzada en 305 iteraciones.
valores:
[0.54200961 0.49878139 0.47067001 0.456824   0.55843582 0.
 0.3583364  0.         0.5917859  0.6430702  0.61519907 0.
 0.         0.74171361 0.86283389 0.        ]
Política óptima (por estado):
[0 3 3 3 0 0 0 0 3 1 0 0 0 2 1 0]
